In [1]:
import sys
import os
sys.path.append(os.path.abspath('../'))

from evaluation import *
from data_loader import *
import random
from datetime import datetime

In [2]:
colors = {
    "DQN": 'darkorange',
    "PPO": 'forestgreen',
    "CMA": 'cyan',
    "ODT": 'blueviolet'
}

In [3]:
experiments = range(1030, 1038)

exp_agent_data = []
exp_station_data = []

agg_counts = []

# REMOVE THIS
algos = ['DQN', 'PPO', 'CMA', 'ODT']

for ind, exp_num in enumerate(experiments):
    config_fname = f'./Exp_{exp_num}/config.yaml'
    
    c = load_config_file(config_fname)
    nn_c = c['nn_hyperparameters']
    federated_c = c['federated_learning_settings']
    algo_c = c['algorithm_settings']
    env_c = c['environment_settings']
    eval_c = c['eval_config']

    ev_info = []

    seed = env_c['seed']

    algorithm_dm = algo_c['algorithm']
    
    def load_from_json_with_error_handling(filepath, columns_specific):
        try:
            return read_csv_data(filepath, columns=columns_specific)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON from {filepath}: {e.msg} at line {e.lineno}, column {e.colno}")
            return None  # Handle the error and return None or an empty object
    
    
    d_base = f"../../../../storage_1/metrics/Exp_{exp_num}"
    
    if not os.path.exists(d_base):
        d_base = f"../metrics/Exp_{exp_num}"
            
    base_path = f"{d_base}/train/metrics"

    print(f'Loading {base_path}_agent_metrics.csv')
    agent_data = load_from_json_with_error_handling(f'{base_path}_agent_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'agent_index', 'distance', 'average_battery'])
    station_data = load_from_json_with_error_handling(f'{base_path}_station_metrics.csv', ['episode', 'timestep', 'done', 'zone', 'aggregation', 'station_index', 'traffic'])
    
    agent_data['seed'] = seed
    station_data['seed'] = seed
    
    agent_data['exp_num'] = exp_num
    station_data['exp_num'] = exp_num
    
    # agent_data['algorithm'] = algorithm_dm
    # station_data['algorithm'] = algorithm_dm
    agent_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
    station_data['algorithm'] = algos[ind % len(algos)] # REMOVE THIS AND UNCOMMENT PREVIOUS LINE!
    
    agent_data['season'] = env_c['season']
    station_data['season'] = env_c['season']
    
    agent_data['num_aggs'] = federated_c['aggregation_count']
    station_data['num_aggs'] = federated_c['aggregation_count']
    
    agent_data['eps_per_agg'] = nn_c['num_episodes']
    station_data['eps_per_agg'] = nn_c['num_episodes']

    exp_agent_data.append(agent_data)
    exp_station_data.append(station_data)

Loading ../../../../storage_1/metrics/Exp_1030/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1031/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1032/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1033/train/metrics_agent_metrics.csv
File ../../../../storage_1/metrics/Exp_1033/train/metrics_agent_metrics.csv not found.
File ../../../../storage_1/metrics/Exp_1033/train/metrics_station_metrics.csv not found.
Loading ../../../../storage_1/metrics/Exp_1034/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1035/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1036/train/metrics_agent_metrics.csv
Loading ../../../../storage_1/metrics/Exp_1037/train/metrics_agent_metrics.csv


In [4]:
# Convert data to DataFrame for easier manipulation
df_agent = pd.concat(exp_agent_data, ignore_index=True)
df_station = pd.concat(exp_station_data, ignore_index=True)

cumulative_agent_df = (
    df_agent
    .groupby(
        ['episode', 'zone', 'aggregation', 'agent_index', 'seed', 'exp_num', 'algorithm', 'season', 'num_aggs', 'eps_per_agg']
    )[['distance', 'average_battery']]
    .sum()
    .reset_index()
)

cumulative_station_df = (
    df_station
    .groupby(
        ['episode', 'zone', 'aggregation', 'station_index', 'seed', 'exp_num', 'algorithm', 'season', 'num_aggs', 'eps_per_agg']
    )['traffic']
    .max()
    .reset_index()
)

# Rename the 'reward' column to 'cumulative_reward' for clarity
cumulative_agent_df.rename(columns={'average_battery': 'battery_used'}, inplace=True)
cumulative_agent_df.rename(columns={'distance': 'distance_traveled'}, inplace=True)

# Get recalculated episodes using (aggregation number * episodes per aggregation) + episode number
cumulative_agent_df['episode'] = cumulative_agent_df['aggregation'] * cumulative_agent_df['eps_per_agg'] + cumulative_agent_df['episode']
cumulative_station_df['episode'] = cumulative_station_df['aggregation'] * cumulative_station_df['eps_per_agg'] + cumulative_station_df['episode']

cumulative_agent_df.head()

,episode,zone,aggregation,agent_index,seed,exp_num,algorithm,season,num_aggs,eps_per_agg,distance_traveled,battery_used
0,0.0,1.0,0.0,0.0,1234,1030,DQN,winter,20,200,36.085532,12682.1696
1,0.0,1.0,0.0,0.0,1234,1034,DQN,winter,13,200,36.452618,20708.8445
2,0.0,1.0,0.0,0.0,1234,1035,PPO,winter,13,200,36.452618,20708.8445
3,0.0,1.0,0.0,0.0,1234,1036,CMA,winter,13,200,36.452618,20708.8445
4,0.0,1.0,0.0,0.0,1234,1037,ODT,winter,13,200,36.452618,20708.8445


In [5]:
cumulative_station_df.head()

,episode,zone,aggregation,station_index,seed,exp_num,algorithm,season,num_aggs,eps_per_agg,traffic
0,0.0,1.0,0.0,0.0,1234,1030,DQN,winter,20,200,0.0
1,0.0,1.0,0.0,0.0,1234,1034,DQN,winter,13,200,0.0
2,0.0,1.0,0.0,0.0,1234,1035,PPO,winter,13,200,0.0
3,0.0,1.0,0.0,0.0,1234,1036,CMA,winter,13,200,0.0
4,0.0,1.0,0.0,0.0,1234,1037,ODT,winter,13,200,0.0


In [6]:
# Get the data for the last episode only
cumulative_agent_df = cumulative_agent_df[cumulative_agent_df['episode'] == cumulative_agent_df['episode'].max()]
cumulative_station_df = cumulative_station_df[cumulative_station_df['episode'] == cumulative_station_df['episode'].max()]

In [8]:
for algo in cumulative_agent_df['algorithm'].unique():
    print(f"===== Algorithm: {algo} =====")

    agent_d = cumulative_agent_df[cumulative_agent_df['algorithm'] == algo]
    station_d = cumulative_station_df[cumulative_station_df['algorithm'] == algo]

    print(f"\tAverage Traffic: {station_d['traffic'].mean():.2f}")
    print(f"\tAverage Distance Travelled: {agent_d['distance_traveled'].mean():.2f}")
    print(f"\tAverage Battery Used (kWh): {agent_d['battery_used'].mean():.2f}")
    print("\t--")
    print(f"\tTraffic Std Dev: {station_d['traffic'].std():.2f}")
    print(f"\tDistance Travelled Std Dev: {agent_d['distance_traveled'].std():.2f}")
    print(f"\tBattery Used Std Dev (kWh): {agent_d['battery_used'].std():.2f}")
    print("")

for season in cumulative_agent_df['season'].unique():
    season_data = cumulative_agent_df[cumulative_agent_df['season'] == season]
    
    for algo in season_data['algorithm'].unique():
        algo_data = season_data[season_data['algorithm'] == algo]

        avg_energy_used = algo_data['battery_used'].mean()
        std_energy_used = algo_data['battery_used'].std()

        print(f"Algorotim: {algo} - Season: {season} \
        - Avg. Energy {avg_energy_used:.2f} - Std. Energy {std_energy_used:.2f}")

===== Algorithm: DQN =====
	Average Traffic: 1.45
	Average Distance Travelled: 37.27
	Average Battery Used (kWh): 16095.85
	--
	Traffic Std Dev: 1.96
	Distance Travelled Std Dev: 1.74
	Battery Used Std Dev (kWh): 3275.36

===== Algorithm: CMA =====
	Average Traffic: 1.47
	Average Distance Travelled: 37.05
	Average Battery Used (kWh): 16803.51
	--
	Traffic Std Dev: 2.04
	Distance Travelled Std Dev: 1.48
	Battery Used Std Dev (kWh): 3378.87

Algorotim: DQN - Season: winter         - Avg. Energy 16095.85 - Std. Energy 3275.36
Algorotim: CMA - Season: winter         - Avg. Energy 16803.51 - Std. Energy 3378.87
